# Polarizability tensor of Water Molecule
With this notebook we would like to extract the polarizability terms for a water molecule in gas-phase, with the aim of analysing the effects of these quantities in implicit solvent environment. In a linear response regime, in a generic environment the electrostatic dipole of a H$_2$O molecule might be separated in two terms:
$$
\vec D = \vec D_0 + \mathbf P \cdot \vec E
$$
where $\vec D_0$ represents the molecular dipole in gas phase. The polarizability tensor $\mathbf P$ represents the response of the molecular dipole to a external field described by the vector $\vec E$. 

## Initialization

In [1]:
# Import the useful classes
import os
import numpy as np
from BigDFT.Calculators import SystemCalculator
from BigDFT.Logfiles import Logfile

In [2]:
# Initialize the calculator
study = SystemCalculator(omp=4, mpi=1)

## Define the initial position and input file

In [3]:
H2O = """\
3 atomic
# xyz dump 
O -2.86857851702e-07 1.76247905159e-15 0.735431116396
H 1.46392609098 2.55351295664e-15 -0.367715450117
H -1.46392580412 2.10942374679e-15 -0.36771566628\
"""

input_base = """\
dft:
  hgrids: 0.35
  ixc: LDA
  gnrm_cv: 1.e-5\
"""

## Run the reference calculation

This allows to find the $\vec{D_0}$.

In [4]:
# Create a new directory and move to it
new_dir = "H2O_new"
current_dir = os.getcwd()
if not current_dir.endswith(new_dir):
    try:
        os.mkdir(new_dir)
    except OSError:
        pass
    os.chdir(new_dir)
# Write the input files on disk
base_name = "H2O_ref"
with open(base_name+".xyz", "w") as f:
    f.write(H2O)
with open(base_name+".yaml", "w") as f:
    f.write(input_base)

In [5]:
# Run the calculation
force_run = False  # If True, force the calculation to run
log_name = "log-{}.yaml".format(base_name)
log_exists = os.path.exists(log_name)
if (log_exists and force_run) or not log_exists:
    study.run(name=base_name)
else:
    print("No need to run the calculation.")
# Keep the logfile in memory
logfiles = {"ref": Logfile(log_name)}

No need to run the calculation.


## Run the calculations with an external electric field

One calculation per electric field direction has to be performed (more precise results would be found by applying both electric field with positive and negative amplitudes). This allows to find the new dipole values $\vec{D}$. It is then easy to determine then the polarizability tensor $\mathbf P$.

In [6]:
def set_inputs(inp, ef_amp):
    """
    Returns the new input files to be run in order to apply
    external electric fields in all directions.
    """
    # Set the additional lines for all the input files
    base_efs = ["\n  elecfield: [{}, 0.0, 0.0]", 
                "\n  elecfield: [0.0, {}, 0.0]",
                "\n  elecfield: [0.0, 0.0, {}]", ]
    ef_inputs = [ef.format(ef_amp) for ef in base_efs]
    # Return the updated base input files (one for each calculation)
    return [input_base + ef_input for ef_input in ef_inputs] 

# Set the input files
efield_amplitude = 1.e-3
efield_inputs = set_inputs(input_base, efield_amplitude)

# Define the base name of the input files and of the logfile
base_names = ["Ex", "Ey", "Ez"]
log_names = ["log-{}.yaml".format(base_name) for base_name in base_names]

In [7]:
# Loop over the electric fields
for i, inp in enumerate(efield_inputs):
    # Write the posinp and input file
    base_name = base_names[i]
    with open(base_name+".yaml", "w") as f:
        f.write(inp)
    with open(base_name+".xyz", "w") as f:
        f.write(H2O)
    # Run the calculation for each electric field
    log_name = log_names[i]
    log_exists = os.path.exists(log_name)
    if (log_exists and force_run) or not log_exists:
        study.run(name=base_name)
    else:
        print("No need to run the calculation.")
    # Keep the logfile in memory
    logfiles[base_name] = Logfile(log_name)

Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n Ex
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n Ey
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n Ez


## Get the polarizability tensor of the water molecule

It is found thanks to the dipoles found for each calculation (see equation in the beginning of the notebook and its implementation below).

In [8]:
def dipole(logfile):
    """
    Returns the dipole found in a given logfile.
    """
    return np.array(logfile.log['Electric Dipole Moment (AU)']['P vector'])

# Compute the polarizability tensor from the dipole moments
# of the previous calculations
D0 = dipole(logfiles["ref"])
D = np.mat(np.zeros(9)).reshape(3,3)
for i, base_name in enumerate(base_names):
    log = logfiles[base_name]
    D[i] = dipole(log) - D0
print D / efield_amplitude

[[  1.08086000e+01  -1.50000000e-04   1.00000000e-02]
 [ -9.80000000e-05   1.02554800e+01   1.00000000e-02]
 [  1.37000000e-04   2.04000000e-03   1.05200000e+01]]
